In [1]:
import pandas as pd
import numpy as np
import itertools, scipy
from itertools import compress
from operator import concat

In [2]:
data_type ='mean' #mat, csv, mix
# dataloader
clinical_data = pd.read_csv('/Users/monoaki/Documents/vs_code/Subjective-Cognitive-Decline-Brain/data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = '/Users/monoaki/Documents/vs_code/Subjective-Cognitive-Decline-Brain/data/dFC matrix matlab/' #raw_data_path = './data/image data/dFC matrix matlab/'
# feature selection process
rank_threshold = 10000
feature_selection_df_savepath = '../feature_selection_result'
class loaders:
    def __init__(self, data_type='mat' ,mat_dtype='FCM_mean'):
        self.data_type = data_type
        if mat_dtype=='mean':
            md ='FCM_mean'
            self.feature_filter_list = []
        elif mat_dtype=='var':
            md = 'FCM_var'
            self.feature_filter_list = []
        else:
            md = None
            self.feature_filter_list = []
        self.mat_dtype = md
        self.concat_mat_array = []
        self.concat_mat_array_mean = []
        self.concat_mat_array_var = []
        

    def feature_filter(self, patient_value, load_path = feature_selection_df_savepath, outcome_name = None, data_type = None):
        feature_rank_df = pd.read_csv('{}/{}_{}_{}.csv'.format(load_path, outcome_name,data_type, 'fc'), index_col=False)
        filter_feature_list = [feature_rank_df['rank'].values<=rank_threshold][0]
        new_patient_value = []
        new_feature = list(compress(feature_rank_df['feature_location'].values, filter_feature_list))
        new_patient_value = list(compress(patient_value, filter_feature_list))
        return np.array(new_patient_value), new_feature

    def mat_process(self, data):
        get_data = []
        for idx, i in enumerate(data):
            if len(i[idx+1::])!=0:
                get_data.append(list(i[idx+1::]))
        get_data = list(itertools.chain(*get_data))
        return np.array(get_data)

    def load_mat(self, data_path=None, outcome_name = "AD8主觀認知障礙[0-1,>=2]"):
        mat = scipy.io.loadmat(data_path)
        if data_type=='mix':
            get_value_m, get_value_v = mat['FCM_mean'], mat['FCM_var']
            get_value_m, get_value_v = self.mat_process(get_value_m), self.mat_process(get_value_v)
            get_value_m, feature_m = self.feature_filter(get_value_m, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type = 'mean')
            get_value_v, feature_v = self.feature_filter(get_value_v, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type = 'var')
            self.feature_filter_list = feature_m + feature_v
            self.concat_mat_array_mean.append(get_value_m)
            self.concat_mat_array_var.append(get_value_v)
        else:
            get_value = mat[self.mat_dtype ]
            get_value = self.mat_process(get_value)
            get_value, feature = self.feature_filter(get_value, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type = data_type)
            self.feature_filter_list = feature
            self.concat_mat_array.append(get_value)

    def callback(self,):
        if data_type=='mix':
            concat_mat_array = np.concatenate((np.array(self.concat_mat_array_var).astype(np.float16),np.array(self.concat_mat_array_mean).astype(np.float16)), axis=1)
            return concat_mat_array, self.feature_filter_list
        else:
            return np.array(self.concat_mat_array).astype(np.float16), self.feature_filter_list

In [3]:
ld = loaders(data_type = 'mat', mat_dtype = data_type)
for next_path in sorted(os.listdir(raw_data_path)):
    if 'mat' in next_path:
        ld.load_mat(data_path = os.path.join(raw_data_path,next_path))

In [4]:
for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
    label_ = clinical_data[outcome_name]
    if 'MOCA' in outcome_name:
        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
    elif 'AD8' in outcome_name:
        label_s = [1 if i>=2 else 0 for i in list(label_)]
        label_ = label_+1
    df = pd.DataFrame(ld.callback()[0], columns = ld.callback()[1])
    df['outcome'] = label_
    df.insert(0, "subj", clinical_data['subj'], allow_duplicates=True)
    print(df.shape)
    df.to_csv(f'../../R/dataset/{outcome_name}_{data_type}_{rank_threshold}_FC.csv', index=False)

(84, 15005)
(84, 15005)
